<a href="https://colab.research.google.com/github/Namsik-Yoon/Namsik-Lab/blob/master/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

pytorch tutorial에 올라와있는 DCGAN예제 실험. 우리는 많은 실제 유명인 사진을 DL Model에게 보여준 후, 새로운 유명인의 사진을 생성할 수 있도록 DL Model(GAN)을 훈련시킬 것입니다. DCGAN 실행을 위한 대부분의 코드는 pytorch tutoral에서 가져온 것이며, 모델의 구현방법 및 작동원리를 설명하고자합니다

# What is GAN?

GAN은 training 데이터의 분포를 알아내어 같은 분포로부터의 새로운 데이터를 생성할 수 있는 DL Model을 학습하는 프레임워크입니다. GAN은 Ian Goodfellow에 의해 2014년 만들어졌으며, 해당 논문은 [Generative Adversarial Nets](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) 에서 볼 수 있습니다.
GAN은 Generator(생성자)와 Discriminator(판별자)라는 두 개의 구분되는 모델로 구성이 되어 있습니다. Generator가 하는 일은 training 데이터와 유사한 '가짜' 이미지를 만들어내는 것입니다. Discriminator가 하는 일은 이미지를 본 뒤, 실제 training 데이터인지 generator로 부터 생성된 가짜 이미지인지를 출력하는 것입니다(아래 그림참조). 트레이닝 동안, generator는 끊임없이 더 나은 가짜 이미지를 생성해 disciminator를 능가하려 노력하는 반면, discriminator는 진짜 이미지와 가짜 이미지를 더 잘 감지하고 분류하기 위해 노력합니다. 이 게임의 균형은 generator가 training 데이터에서 꺼내온 듯한 완벽한 가짜 이미지를 만들어 내고 있을 때이고, discriminator는 항상 generator의 결과가 진짜인지 가짜인지 50%의 신뢰도를 가지고 추측하도록 둡니다.

![대체 텍스트](https://img1.daumcdn.net/thumb/R720x0.q80/?scode=mtistory2&fname=http%3A%2F%2Fcfile3.uf.tistory.com%2Fimage%2F9928E6375B75872D170654)

Discriminator를 시작으로 튜토리얼을 시작할 텐데, 몇 가지 표기법을 정의해봅시다. 지금부터 'x'를 이미지를 대표하는 데이터라고 부릅시다. D(x)는 x가 generator가 아닌 training 데이터에서 나왔을 ***확률***을 출력하는 discriminator 네트워크이다. 우리는 이미지 데이터를 다루기 때문에 D(x)에 대한 입력의 크기(size)는 3 * height * width입니다. 직관적으로 D(x)는 x가 training 데이터에서 나왔을 때 높고, generator로부터 만들어졌을 때 낮다. D(x)는 전통적인 이진 분류기 (binary classifier)로 생각할 수 있다.

Generator를 위한 표기법으로는,  표준정규분포로부터 추출된 잠재 공간(latent space) 벡터를 'z'라고 할때, G(z)는 잠재 벡터 z를 데이터 공간(data-space, 이미지)으로 매핑시켜주는 generator 함수이다. G의 목표는 training data의 p_data 분포를 추정하여, 그 추정 분포(p_g)로부터 가짜 샘플을 생성하는 것이다.

따라서, D(G(z)) -> 생성모델이 만든 이미지를 판별모델이 진짜 이미지라고 판별할 확률을 의미한다. D는 진짜와 가짜를 정확하게 구분하는 확률(logD(x))을 최대로, G는 진짜와 가짜를 반대로 구분하는 확률(log(1-D(G(x))))을 최대로 하는 minmax게임이다.
![대체 텍스트](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2Fcj4YMl%2FbtqCinxOcMb%2FlvmWH1KmDyPxz3SKCCtv11%2Fimg.png)

이론상으로 이 minmax게임의 해결책은 p_g = p_data (추정 분포가 train_data의 분포)일때이며, D는 인풋이 진짜인지 가짜인지를 랜덤하게 추측한다(?,추가공부 필요). 

# What is DCGAN?
DCGAN은 위에서 설명한 GAN의 확장으로, generator와 discriminator에서 각각 Convolution 레이어와 Convolutional-transpose 레이어(Deconvolution이라고도 하며 쉽게 얘기하면 padding을 늘려서 input의 차원을 늘리는 기법; 하단 이미지 참조)를 사용한다는 점에서 차이가 있다. DCGAN은 [Unsupervised Representation Learning with Deep Convolution Generative Adversarial Networks](https://arxiv.org/pdf/1511.06434.pdf)논문에서 Radford et.al.에 의해 처음 발표되었다. Discriminator는 strided convolution 레이어, Batch Norm 레이어 그리고 Leaky ReLU 활성화 함수로 구성되어 있다. 인풋은 3*64*64 이미지이며 아웃풋은 인풋이 실제 데이터 분포(* = training data)에서 가져온 데이터일 확률입니다. Generator는 convolutional-transpose 레이어, Batch Norm 레이어, 그리고 ReLU 활성화 함수로 구성된다.

![대체 텍스트](https://ifh.cc/g/nD0Cm.png)

![대체 텍스트](https://ifh.cc/g/d3GPq.png)

In [0]:
import torch

input = torch.rand(1,1,3,3)

ConvT = torch.nn.ConvTranspose2d(1,1,kernel_size=3,stride=2,padding=1,output_padding=1)

output = ConvT(input)

print(f'input size = {input.size()}')
print(f'output size = {output.size()}')

generator의 인풋은 표준정규분포에서 추출된 잠재벡터 z이고, 아웃풋은 3 * height * width 이미지이다. convolutional-transpose 레이어는 잠재 벡터를 이미지와 같은 모양으로 변환하며 논문에서는 optimizer를 어떻게 설정하는지, loss function을 어떻게 계산하는지, model의 weight는 어떻게 초기화하는지 등등 몇가지 팁을 제시한다. 이 내용들은 뒤에 오는 세션들에서 설명할 예정이다.

![대체 텍스트](https://ifh.cc/g/yT77H.png)


In [0]:
%matplotlib inline

In [0]:

#%matplotlib inline
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [0]:
!gdown --id 19m3N3lD4ILVxc_lADkM_2-EGVZUiiSWn --output img_align_celeba.zip
!unzip /content/img_align_celeba.zip

In [0]:
!mkdir img
!mv "/content/img_align_celeba" "/content/img"

#Inputs
실행을 위해 몇가지 inputs를 정의하자

*   dataroot - 데이터세트 폴더의 루트 경로

*   workers - Dataloader를 이용해 데이터를 로드하기 위해 사용되는 스레드 수

*   batch_size - 학습에서 사용되는 배치 크기. 논문에서는 128

*  image_size - 학습에서 사용되는 이미지 크기. (모든 이미지가 이 사이즈에 맞게 resize 된다.) 디폴트 값은 64 * 64. 다른 크기로 이용하기 위해서는 D(Discriminator)와 G(Generator)의 구조를 바꾸어야 한다.
*   nc - 인풋 이미지의 색상 채널 수. 컬러 이미지는 이 값이 3이다. (*흑백 이미지의 경우 1)


*   nz - 잠재 벡터의 길이(크기)


*   ngf - Generator를 거치는 피쳐 맵의 크기(걍 채널수인데 이걸 왜 미리 설정하는지...)


*   gdf - Discriminator를 거치는 피쳐 맵의 크기(?)

*   num_epochs - training epoch의 수. 오랜 시간 학습하는 것이 더 좋은 결과를 낳지만, 더 오랜 시간이 걸릴 수 있다.
*   lr - 학습에서 사용되는 learning rate. 논문에서는 0.0002


*   beta1 - Adam optimizer를 위한 하이퍼파라미터이다. 논문에서는 0.5
*   ngpu - 가능한 gpu의 수. 이 값이 0이라면 CPU 모드에서 작동할 것이다. 0보다 더 큰 수일 경우, 숫자만큼의 GPU에서 작동할 것이다.






In [0]:
dataroot = "/content/img"
workers = 4
batch_size = 4
image_size = 64
nc = 3
nz = 100
ngf = 64
ndf = 64
num_epochs = 100
lr = 0.0002
beta1 = 0.5
ngpu = 1

In [0]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

# Implementation
우리의 인풋 파라미터와 준비된 데이터를 가지고 이제 본격적인 작업을 한다. 우리는 weight initialization 전략을 시작으로, generator, discriminator, loss functions, training loop에 대해 자세히 살펴보고자 한다.


## Weight Initialization
DCGAN 논문에서 저자는 모든 모델의 weight는 평균 0(당연한거 아닌가?), 표준편차 0.02의 정규 분포에서 랜덤하게 초기화 되어야한다고 명시했다. 'weights_init' 함수는 초기화된 모델을 인풋으로 가진다. 그리고 모든 convolutional, convolutional-transpose, 그리고 batch-normalization 레이어를 이 기준에 맞게 재초기화한다. 이 함수는 초기화된 직후의 모델에 적용된다.

In [0]:
def weights_init(m):
  classname = m.__class__.__name__

  if classname.find('Conv') != -1:
    nn.init.normal_(m.weight.data, 0.0,0.02)
  
  elif classname.find('BatchNorm') != -1:
    nn.init.normal_(m.weight.data,1.0,0.02)
    nn.init.constant_(m.bias.data,0)

## Generator
Generator, G는 잠재 공간의 벡터(z)를 데이터 공간으로 매핑하게  디자인되었다. 우리의 데이터는 이미지이기 때문에, z를 데이터 공간으로 변환하는 것은 궁극적으로 학습 데이터와 같은 사이즈(3*64*64)의 RGB 이미지를 만드는 것과 같다. 실제로, 이것은 2d batch norm 레이어와 Relu 활성화 함수와 짝을 이루는 2차원의 strided convolutional transpose layer들을 통해 이루어진다. 인풋 데이터의 범위를 [-1,1]로 조정하기 위해 generator의 아웃풋은 tan h 함수를 거친다. Conv-transpose 레이어 뒤에 Batch norm function이 위치하는 것이 이 논문의 아주 중요한 내용(critical distribution)이다. 이 레이어들은 학습하는 동안에 그래디언트가 흐를 수 있게 돕는다. 논문에서의 generator 이미지는 아래와 같다.


ps. 그림이 좀 잘못된게 코드로 따져본다면 잠재공간벡터 z의 사이즈는 100* 1 *1 이므로 세우지말고 눕혀야한다

![대체 텍스트](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2Fcyg8JP%2FbtqCgokAef9%2FxEr4tbJd3t7zw4kkgyiKJK%2Fimg.png)

인풋 섹션(nz, ngf 및 nc)에서 인풋에 대해 설정한 것이 generator의 코드 구조에 어떻게 영향을 미치는지 주목해보자. nz는 인풋 벡터 z의 길이(차원)이고, ngf는 generator를 통해 순전파(propagate)되는 피쳐 맵의 사이즈와 관련이 있다. 그리고 nc는 아웃풋 이미지의 채널의 수(RGB 이미지의 경우 3)이다. 아래는 generator에 대한 코드이다.

In [0]:
# Generator

class Generator(nn.Module):
  def __init__(self,ngpu):
    super(Generator, self).__init__()
    self.ngpu = ngpu
    self.main = nn.Sequential(
        # input : z 벡터 [100*1*1]
        nn.ConvTranspose2d(in_channels=nz,out_channels=ngf*8,kernel_size=4,stride=1,padding=0, bias=False),
        nn.BatchNorm2d(ngf*8),
        nn.ReLU(True),
        # state size. (ngf*8)*4*4
        nn.ConvTranspose2d(ngf*8,ngf*4,4,2,1,bias=False),
        nn.BatchNorm2d(ngf*4),
        nn.ReLU(True),
        # (ngf*4)*8*8
        nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1,bias=False),
        nn.BatchNorm2d(ngf*2),
        nn.ReLU(True),
        # (ngf*2)*16*16
        nn.ConvTranspose2d(ngf*2,ngf,4,2,1,bias=False),
        nn.BatchNorm2d(ngf),
        nn.ReLU(True),
        # ngf * 32 * 32
        nn.ConvTranspose2d(ngf,nc,4,2,1,bias=False),
        nn.Tanh()
        # nc * 64 *64
    )

  def forward(self,input):
    return self.main(input)

이제, 우리는 generator를 인스턴스화하고 weights_init 함수를 적용할 수 있다. Generator 객체가 어떻게 구조화되었는지 확인해보아라.

In [0]:
netG = Generator(ngpu).to(device)

if (device.type=='cuda') and (ngpu>1):
  netG = nn.DataParallel(netG, list(range(ngpu)))

netG.apply(weights_init)

print(netG)

## Discriminator
언급한 것과 같이 discriminator, D는 이미지를 인풋으로, 인풋 이미지가 진짜일 확률을 아웃풋으로 하는 이진 분류(binary classification) 네트워크이다. D는 3*64*64 이미지를 입력받아, 여러 개의 Conv2d 레이어, BatchNorm2d 레이어, LeakyReLU 레이어를 지나고, 최종적으로 시그모이드 함수를 거친 확률 값을 아웃풋을 출력한다. 이 문제(태스크)를 위해 필요하다면 더 많은 레이어들로 확장될 수 있지만, strided convolution, Batch Norm, LeakyReLUs를 사용하는 데에도 의미가 있다. DCGAN 논문은 네트워크가 자체적인 풀링 기능을 학습할 수 있다는 점에서 downsample을 위한 풀링보다는 strided convolution을 사용하는 것이 좋다고 이야기한다.  또한 G와 D의 학습과정에서 아주 중요한 그래디언트의 흐름이 batch norm과 leaky relu 함수를 통해 잘 일어날 수 있다고 이야기한다.

In [0]:
# Discriminator

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

이제 Generator와 마찬가지로 discriminator를 만들고, weights_init 함수를 적용한 뒤 모델의 구조를 확인할 수 있다.

In [0]:
netD = Discriminator(ngpu).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

netD.apply(weights_init)

print(netD)

## Loss Functions and optimizers

우리는 D와 G가 loss function과 optimizer를 통해 어떻게 학습할지를 지정할 수 있다. 우리는 파이토치에서 아래와 같은 식으로 정의된 Binary Cross Entropy loss(BCELoss)를 이용할 것이다. 

![대체 텍스트](https://k.kakaocdn.net/dn/WofGK/btqCe5r3Wfn/5N9CTaurZ87aYRQkwQq0Ak/img.png)

이 함수가 목표 함수의 두 로그 부분(log(D(x))와 log(1-D(G(z)))를 어떻게 계산하는지 주목하자. 우리는 BCE 방정식의 어떤 부분을 y 인풋과 함께 사용할지 지정할 수 있다.  이 내용은 뒤에 올 트레이닝 루프에서 이루어지지만, y(즉 GT 라벨)을 변해 계산하고자 하는 요소를 어떻게 고를 수 있는지를 이해하는 것이 중요하다. 

다음으로, 우리는 진짜 사진에 대한 라벨을 1, 가짜 사진에 대한 라벨을 0으로 정의하자. 이 라벨은 D와 G의 loss를 계산할 때 이용되며 본래 GAN 논문에서 이용되던 방식이다. 최종적으로, 우리는 D와 G 각각을 위한 두 개의 구분된 optimizer를 둔다. DCGAN 논문에서는 두 optimizer 모두 Adam optimizer이며, learning rate = 0.0002 , Beta1 = 0.5이다. Generator의 학습 과정을 추적하기 위해, 우리는 가우시안 분포에서 온 고정된 배치 사이즈의 잠재 벡터를 생성할 것이다. 학습 과정에서 우리는 주기적으로 이 fixed_noise 벡터를 G에 입력해, 이 벡터로부터 생성된 이미지를 확인할 것이다.

In [0]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(64,nz,1,1,device=device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1,0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1,0.999))

# Training
이제 우리는 GAN 프레임워크의 모든 부분을 정의했고 이제 모델을 훈련시킬 수 있다. GAN을 훈련시키는 것은 일종의 예술 형태임을 명심하라. 잘못된 하이퍼 파라미터 설정은 무엇이 잘못되었는지 설명이 거의 되지 않으채로 모드 붕괴(mode collapse)로 이어지기 때문이다. 여기서는 Goodfellow의 논문에 나오는 알고리즘 1을 거의 따르면서 ganhacks에 나타난 좋은 사례를 따를 것이다.  다시 말해, 우리는 "진짜 이미지와 가짜 이미지에 대한 각기 다른 미니 배치를 구성"할 것이고 또한 G의 목적 함수를 log(D(G(z))를 최대화하는 것으로 조정할 것이다. 학습은 두 개의 메인 파트로 나뉜다. 파트 1은 D를 업데이트하는 것이고, 파트 2는 G를 업데이트하는 것이다.

## Part 1 - Train the Discriminator

Discriminator의 목적은 인풋으로 주어진 진짜 혹은 가짜 이미지를 올바르게 분류할 확률을 키우는 것임을 기억해야 한다. Goodfellow의 관점에 따라 우리는 "discriminator를 stochastic gradient를 상승시킴으로써 업데이트하기"를 바란다. 실제로 우리는 log(D(x))+log(1-D(G(z)))를 극대화하고자 한다. Ganhack에서의 제안한 미니 배치로 인해, 우리는 두 단계로 나누어 계산을 할 것이다. 첫째, 트레이닝 세트로부터 진짜 샘플을 가져와 배치를 구성한 뒤 D를 거치게 해 loss를 계산.(logD(x)) 그리고 거꾸로 그래디언트를 계산.(* = backpropa) 둘째, 우리는 현재의 generator로 만든 가짜 샘플로 배치를 구성한 뒤 D를 거치게 한뒤, loss를 계산하고(log(1-D(G(z))) 거꾸로 그래이던트를 계산한다. 이제 모든 진짜 이미지, 가짜 이미지 배치에서의 그래디언트를 모두 합해 discriminator의 opitimizer의 step을 호출한다. (* step() 함수를 의미하는 것 같음. )

## Part 2 - Train the Generator
본 논문에서 기술된 것과 같이, 우리는 더 나은 가짜 이미지를 만들기 위한 노력으로  log(1-D(G(z)))를 최소화함으로써 generator를 훈련시키고자 한다. 언급한 것과 같이, 이것은 특히나 학습 초기에 충분한 그래디언트를 제공하지 못한다는 사실이 Goodfellow에 의해 보여졌다. 이를 우리는 log(D(G(z)))를 최대화하는 방향으로 수정하였다. 이것을 다음과 같은 코드를 통해 달성한다 : Generator의 아웃풋을 ~, 진짜 라벨 GT를 이용해 G의 loss를 계산, 거꾸로 G의 그래디언트 계산, 그리고 최종적으로 optimizer step을 이용해 G의 파라미터를 업데이트한다. loss function에 진짜 라벨 GT를 사용하는 것이 직관적이지 않다고 보일 수 있지만, 이것은 우리가 정확히 원하던 BCELoss의 log(x) 파트를 사용할 수 있도록 합니다.

최종적으로, 우리는 몇몇의 통계량을 보고할 것이고, 각각의 epoch이 끝날 때마다 G의 훈련과정을 시각적으로 확인하기 위해 fixed_noise 배치를 generator에 입력할 것입니다. 학습 과정에서 보고될 통계량은 다음과 같습니다.


*   Loss_D - 모든 진짜, 가짜 이미지 배치에 대한 loss를 합한 값

*   Loss_G - log(D(G(z)))로 계산된 generator loss
*   D(x) - 진짜 이미지 배치에 대한 discriminator의 평균 아웃풋. 1에 가깝게 시작해서, G가 나아질수록 이론적으로 0.5에 수렴하게 된다. 


*   D(G(z) - 가짜 이미지 배치에 대한 discriminator의 평균 아웃풋. 첫 번째 값은 D가 업데이트되기 전, 두 번째 값은 D가 업데이트된 후의 숫자이다. 이 숫자들은 0에 가깝게 시작해서 G가 나아질수록 0.5에 가깝게 된다.



In [0]:
# Training

img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")

#for each epoch
for epoch in range(num_epochs):
  # for each batch
  for i, data in enumerate(dataloader,0):
    
    # Update D

    # train with all-real batch
    netD.zero_grad()
    real_cpu = data[0].to(device)
    b_size = real_cpu.size(0)
    label = torch.full((b_size,), real_label, device=device)

    output = netD(real_cpu).view(-1)
    errD_real = criterion(output,label)
    errD_real.backward()
    D_x = output.mean().item()

    # train with all-fake batch
    noise = torch.randn(b_size,nz,1,1,device=device)
    fake = netG(noise)
    label.fill_(fake_label)

    output = netD(fake.detach()).view(-1)
    errD_fake = criterion(output,label)
    errD_fake.backward()
    D_G_z1 = output.mean().item()
    errD = errD_real + errD_fake

    optimizerD.step()

    # Update G
    netG.zero_grad()
    label.fill_(real_label)
    
    output = netD(fake).view(-1)
    errG = criterion(output,label)
    errG.backward()
    D_G_z2 = output.mean().item()
    
    optimizerG.step()

    if i % 50 == 0:
      print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
      
      G_losses.append(errG.item())
      D_losses.append(errD.item())

      if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

      iters += 1

In [0]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [0]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=10, repeat_delay=10, blit=True)

HTML(ani.to_jshtml())

In [0]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()